# RDD 내부동작 원리

1. 특정 컬럼의 고유한 값의 개수를 세기
2. A로 시작하는 것들 찾기
3. 스크린에 결과 출력하기

    - 위 과정을 코드로 나눠보면
    1. map(lambda v:(v, 1))를 이용, 스파크가 A를 포함하는 단어를 모음
    2. .filter(lambda val: bal.startwith('A'))를 사용해서 A로 시작하는 단어를 필터링
    3. .reduceByKey(operator.add)를 호출
    4. 트랜스폼
    5. collect()함수를 호출.
    6. 액션

In [1]:
import findspark, pyspark
findspark.find()

'C:\\Bigdata\\spark-2.4.5-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

conf  = pyspark.SparkConf().setAppName('appName').setMaster('local[2]')
sc    = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [3]:
data = sc.parallelize([
    ('Amber',22), ('Alfred',23), ("Skye",4), ("Albert",12), ("Amber", 9)
])

In [4]:
data_from_file = sc.textFile("../../data/RDD_example/VS14MORT.txt.gz", 4)

In [5]:
type(data_from_file)

pyspark.rdd.RDD

## .textFile()
1. 데이터 파일의 위치(경로)
1. 파티션의 갯수( 노드/클러스터당 2~4개 정도의 파티션으로 관리)

    - 지원되는 파일 시스템
        - Windows ( NTFS, FAT )
        - MAC ( HFS+ )
        - 분산파일시스템( HDFS, S3, Cassandra )

    - 경로에는 다음과 같은 특수문자(대괄호)는 사용할 수 없다.  
        - []

    - 지원되는 텍스트포맷
        - CSV, JSON, HIVE 테이블, JDBC 드라이버, parquet, 압축 Gzip, tar.gz, gz, zip
    
    - 리스트, 튜플, 딕셔너리를 동시에 구성할 수 있다

In [6]:
data_heterogenous = sc.parallelize([
    ("Ferrari", "fast"), {"Porsche":100000}, ['Spain', 'visited', 4504]
]).collect()

In [7]:
data_heterogenous[1]['Porsche']

100000

In [30]:
data_from_file.take(1)

['                   1                                          2101  M1087 432311  4M4                2014U7CN                                    I64 238 070   24 0111I64                                                                                                                                                                           01 I64                                                                                                  01  11                                 100 601']

In [31]:
#  extractInformation 함수 관련
    '''
        Input record schema
        schema: n-m (o) -- xxx
            n - position from
            m - position to
            o - number of characters
            xxx - description
        1. 1-19 (19) -- reserved positions
        2. 20 (1) -- resident status
        3. 21-60 (40) -- reserved positions
        4. 61-62 (2) -- education code (1989 revision)
        5. 63 (1) -- education code (2003 revision)
        6. 64 (1) -- education reporting flag
        7. 65-66 (2) -- month of death
        8. 67-68 (2) -- reserved positions
        9. 69 (1) -- sex
        10. 70 (1) -- age: 1-years, 2-months, 4-days, 5-hours, 6-minutes, 9-not stated
        11. 71-73 (3) -- number of units (years, months etc)
        12. 74 (1) -- age substitution flag (if the age reported in positions 70-74 is calculated using dates of birth and death)
        13. 75-76 (2) -- age recoded into 52 categories
        14. 77-78 (2) -- age recoded into 27 categories
        15. 79-80 (2) -- age recoded into 12 categories
        16. 81-82 (2) -- infant age recoded into 22 categories
        17. 83 (1) -- place of death
        18. 84 (1) -- marital status
        19. 85 (1) -- day of the week of death
        20. 86-101 (16) -- reserved positions
        21. 102-105 (4) -- current year
        22. 106 (1) -- injury at work
        23. 107 (1) -- manner of death
        24. 108 (1) -- manner of disposition
        25. 109 (1) -- autopsy
        26. 110-143 (34) -- reserved positions
        27. 144 (1) -- activity code
        28. 145 (1) -- place of injury
        29. 146-149 (4) -- ICD code
        30. 150-152 (3) -- 358 cause recode
        31. 153 (1) -- reserved position
        32. 154-156 (3) -- 113 cause recode
        33. 157-159 (3) -- 130 infant cause recode
        34. 160-161 (2) -- 39 cause recode
        35. 162 (1) -- reserved position
        36. 163-164 (2) -- number of entity-axis conditions
        37-56. 165-304 (140) -- list of up to 20 conditions
        57. 305-340 (36) -- reserved positions
        58. 341-342 (2) -- number of record axis conditions
        59. 343 (1) -- reserved position
        60-79. 344-443 (100) -- record axis conditions
        80. 444 (1) -- reserve position
        81. 445-446 (2) -- race
        82. 447 (1) -- bridged race flag
        83. 448 (1) -- race imputation flag
        84. 449 (1) -- race recode (3 categories)
        85. 450 (1) -- race recode (5 categories)
        86. 461-483 (33) -- reserved positions
        87. 484-486 (3) -- Hispanic origin
        88. 487 (1) -- reserved
        89. 488 (1) -- Hispanic origin/race recode
     '''

IndentationError: unexpected indent (<ipython-input-31-3e27bc338e13>, line 2)

In [32]:
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [
         2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,
         19,21,22,23,24,25,27,28,29,30,32,33,34,
         36,37,38,39,40,41,42,43,44,45,46,47,48,
         49,50,51,52,53,54,55,56,58,60,61,62,63,
         64,65,66,67,68,69,70,71,72,73,74,75,76,
         77,78,79,81,82,83,84,85,87,89
    ]
    record_split = re\
        .compile(
            r'([\s]{19})([0-9]{1})([\s]{40})([0-9\s]{2})([0-9\s]{1})([0-9]{1})([0-9]{2})' + 
            # 공백포함 문자 19자리, 숫자 한자리, ..
            r'([\s]{2})([FM]{1})([0-9]{1})([0-9]{3})([0-9\s]{1})([0-9]{2})([0-9]{2})' + 
            r'([0-9]{2})([0-9\s]{2})([0-9]{1})([SMWDU]{1})([0-9]{1})([\s]{16})([0-9]{4})' +
            r'([YNU]{1})([0-9\s]{1})([BCOU]{1})([YNU]{1})([\s]{34})([0-9\s]{1})([0-9\s]{1})' +
            r'([A-Z0-9\s]{4})([0-9]{3})([\s]{1})([0-9\s]{3})([0-9\s]{3})([0-9\s]{2})([\s]{1})' + 
            r'([0-9\s]{2})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})([A-Z0-9\s]{7})' + 
            r'([A-Z0-9\s]{7})([\s]{36})([A-Z0-9\s]{2})([\s]{1})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})' + 
            r'([A-Z0-9\s]{5})([A-Z0-9\s]{5})([A-Z0-9\s]{5})([\s]{1})([0-9\s]{2})([0-9\s]{1})' + 
            r'([0-9\s]{1})([0-9\s]{1})([0-9\s]{1})([\s]{33})([0-9\s]{3})([0-9\s]{1})([0-9\s]{1})')
    try:
        rs = np.array(record_split.split(row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs
#     return record_split.split(row)

In [35]:
data_from_file_conv = data_from_file.map(extractInformation)
data_from_file_conv.map(lambda row: row).take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## 스파크의 2가지 모드

- 스파크는 두가지 모드로 동작가능
    - 로컬 모드
        - 파이썬을 실행시키는 것과 다르지 않다
        - 바뀐것은 대부분 구문상의 것들과 데이터와 코드가 분리된 원커 프로세스 사이에서 복사 될 수 있다는 약간의 구조적인 차이
    - 클러스터 모드
        - UDF (User Defined Function)사용시 특별한 주의없이 같은 코드를 클러스터 모드에서 실행하면 골치아픈 일들이 많이 생기니 주의
        - 꼭 필요시에만 UDF를 사용 권장
    
    

## Transformations
- .map()
    - map() 함수는 가장많이 사용하는 함수, 이 함수는 RDD의 각 엘리먼트에 적용됨
    

In [36]:
data_2014 = data_from_file_conv.map( lambda row: int(row[16]) )
data_2014.take(10)

[2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, 2014, -99]

## .filter()
- 이 함수를 사용해서 데이터셋으로 부터 특정조건에 맞는 엘리먼트를 선택가능

In [47]:
data_filtered = data_from_file_conv.filter(
    lambda row: row[5] == 'F' and row[21] == '0' )

In [48]:
data_filtered.count() # 시간이 꽤 걸림

6

In [49]:
data_filtered.take(1)

[array(['2', '12', ' ', '0', '07', 'F', '1', '030', ' ', '32', '12', '05',
        '  ', '1', 'D', '6', '2014', 'N', '1', 'U', 'Y', '0', '9', 'X44 ',
        '420', '122', '   ', '39', '05', '11T391 ', '12X44  ', '13T401 ',
        '14T424 ', '61F199 ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '05',
        'X44  ', 'F199 ', 'T391 ', 'T401 ', 'T424 ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [50]:
data_filtered1 = data_from_file_conv.filter( 
    lambda row: row[16]=='2014' and row[21]=='0' )

In [51]:
data_filtered1.count()

22

In [52]:
data_filtered1.take(1)

[array(['2', '12', ' ', '0', '07', 'F', '1', '030', ' ', '32', '12', '05',
        '  ', '1', 'D', '6', '2014', 'N', '1', 'U', 'Y', '0', '9', 'X44 ',
        '420', '122', '   ', '39', '05', '11T391 ', '12X44  ', '13T401 ',
        '14T424 ', '61F199 ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '05',
        'X44  ', 'F199 ', 'T391 ', 'T401 ', 'T424 ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## .flatMap()
- map()함수와 비슷하게 동작. 
- 리스트가 아닌 평면화된 결과를 리턴한다

In [53]:
data_2014_flat = data_from_file_conv.flatMap( 
    lambda row: (row[16], int(row[16])+1) )

In [54]:
data_2014_flat.take(10)

['2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015, '2014', 2015]

## .distinct()
- 이 함수는 특정컬럼에서의 중복된 값을 제거해 고유한 값을 리스트로 리턴한다
- 데이터를 섞는 함수는 많은 자원을 사용하는 함수이다. 꼭 필요한 경우에만 사용
- 한 메모리에 올라와 있는것을 찾는 판다스와 달리 분산저장된곳들에서 찾아오는 스파크

In [55]:
distinct_gender = data_from_file_conv.map( 
    lambda row: row[5] ).distinct().collect()

In [56]:
distinct_gender

['M', 'F', '-99']

## .sample()
- 함수는 데이터셋으로부터 임의로 추출된 샘플을 리턴한다
    1. 중복허용 여부
    1. 리턴할 데이터셋과 전체 데이터셋 간의 크기 비율
    1. 임의의 숫자를 생성하기 위한 시드값

In [58]:
fraction = 0.1
data_sample = data_from_file_conv.sample( False, fraction, 555)
data_sample.take(1)

[array(['1', '  ', '2', '1', '01', 'M', '1', '058', ' ', '37', '17', '08',
        '  ', '4', 'D', '3', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I250',
        '214', '062', '   ', '21', '03', '11I250 ', '61I272 ', '62E669 ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '03',
        'I250 ', 'E669 ', 'I272 ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

In [59]:
print("Original dataset: {0}, sample: {1}".format( 
                    data_from_file_conv.count(), data_sample.count()) )

Original dataset: 2631171, sample: 262959


## .leftOuterJoin()
- 왼쪽기준 두개 데이터셋 합침

In [64]:
rdd1 = sc.parallelize([ ('a', 1), ('b', 4), ('c', 10) ])
rdd2 = sc.parallelize([ ('a', 4), ('a', 1), ('b', 6), ('d', 15) ])
rdd3 = rdd1.leftOuterJoin(rdd2)
rdd3.take(5)

[('b', (4, 6)), ('c', (10, None)), ('a', (1, 4)), ('a', (1, 1))]

In [65]:
rdd4 = rdd1.join(rdd2)
rdd4.collect()

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1))]

In [66]:
rdd5 = rdd1.intersection(rdd2)
rdd4.collect()

[('b', (4, 6)), ('a', (1, 4)), ('a', (1, 1))]

## .repartition()
- 데이터셋 재파티션하면 데이터가 나눠지는 파팃녀의 개수가 바뀐다
- 부하 많이가는 작업


In [67]:
rdd1 = rdd1.repartition(4)
len(rdd1.glom().collect())

4

In [68]:
help(rdd1.glom)

Help on method glom in module pyspark.rdd:

glom() method of pyspark.rdd.RDD instance
    Return an RDD created by coalescing all elements within each partition
    into a list.
    
    >>> rdd = sc.parallelize([1, 2, 3, 4], 2)
    >>> sorted(rdd.glom().collect())
    [[1, 2], [3, 4]]



## Actions
- 액션은 데이터셋에서 스케쥴된 작업을 수행한다

- .take()
    - map()함수 처럼 역시 가장 유용한 함수이다
        - 이 함수는 하나의 파티션에서 가장 위에있는 n행을 리턴한다
    - RDD 전체를 리턴하는 .collect()보다 자주 사용함
        - 큰 데이터셋일수록 중요하다

In [69]:
data_first = data_from_file_conv.take(1)
data_first

[array(['1', '  ', '2', '1', '01', 'M', '1', '087', ' ', '43', '23', '11',
        '  ', '4', 'M', '4', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I64 ',
        '238', '070', '   ', '24', '01', '11I64  ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '01',
        'I64  ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## .takeSample()
- 데이터로부터 임의의 샘플을 얻고 싶다면 .takeSample() 이용
    - 이 함수는 3개의 파라미터
        1. 중복허용 여부
        1. 리턴되는 데이터 갯수
        1. 랜덤 시드값

In [70]:
data_take_sample = data_from_file_conv.takeSample(False, 1, 557)
data_take_sample

[array(['2', '  ', '2', '1', '09', 'M', '1', '075', ' ', '41', '21', '10',
        '  ', '1', 'M', '5', '2014', 'U', '7', 'C', 'N', ' ', ' ', 'I519',
        '233', '068', '   ', '22', '02', '11I469 ', '21I519 ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '       ',
        '       ', '       ', '       ', '       ', '       ', '02',
        'I519 ', 'I469 ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '     ',
        '     ', '     ', '     ', '     ', '     ', '     ', '01', ' ',
        ' ', '1', '1', '100', '6'], dtype='<U40')]

## .reduce()
- 특정함수를 사용해 RDD의 갯수를 줄인다


In [71]:
rdd1.map(lambda row: row[1]).reduce(lambda x, y: x+y)

15

- rdd1의 값 리스트를 map() transformation을 이용해 생성했고 그 결과를 처리하기 위해 .reduce()함술 이용
- .reduce()함수는 각각의 파티션에서 합계함수를 수행하고 마지막집계가 수행되는 드라이버 노드에 그 합계를 리턴

## 유의사항
- 리듀셔로 전달되는 함수는 결합법칙과 교환법칙이 성립해야 한다
    - 앨리먼트의 순서가 바뀌어도 결과에는 영향을 주지 않아야함
        - (5+2)-3 = 5+(2-3)
    - 피연산자의 순서과 바뀌어도 결과는 같아야 한다
        - 2+3 = 3+2

In [79]:
data_reduce = sc.parallelize([1, 2, .5, .1, 5, .2], 1)
data_reduce.take(7)

[1, 2, 0.5, 0.1, 5, 0.2]

- 데이터를 현재 결과와 다음 결과를 나눈는 방법으로 리듀스하면 예상되는 결과는 10이 나온다(파티션 하나일때)

In [73]:
works = data_reduce.reduce(lambda x,y: x/y)
works

10.0

In [76]:
data_reduce1 = sc.parallelize([1, 2, .5, .1, 5, .2], 3)
data_reduce1.take(10)

[1, 2, 0.5, 0.1, 5, 0.2]

In [81]:
data_reduce1.reduce(lambda x,y: x/y)

0.004

- 키 값을 기반으로 리듀스 하기


In [84]:
data_key = sc.parallelize([ ('a', 4), ('b', 3), ('c', 2), 
                           ('a', 8), ('d', 2), ('b', 1), ('d', 3) ], 4)
data_key.reduceByKey( lambda x,y: x+y ).collect()
# a=4+8, b=3+1, c=2, d=2+3

[('b', 4), ('c', 2), ('a', 12), ('d', 5)]

## .count()
- 전체 데이터셋을 드라이버로 옮기지 않는다
    - len(data_reduce.collect()) 와 동일한 결과이나
    - 연산결과 뿐만 아니라 데이터 전체를 가져오는 차이존재.
    - 양에 따라 문제 발생 여지.

In [86]:
data_reduce.count()

6

- 데이터 셋이 key:value 형태로 있을경우 고유키의 수를 구하기 위해 countByKey() 사용

In [88]:
data_key.countByKey().items()

dict_items([('a', 2), ('b', 2), ('c', 1), ('d', 2)])

## .saveAsTextFile()
- 각 파티션을 분리된 파일에 저장

In [89]:
data_key.saveAsTextFile("../../data/RDD_example/PySpark_data_key.txt")

- 모든 행이 스트링으로 인식, 뒤쪽부터 읽고 싶으면 뒤쪽부터 파싱해야 함

In [97]:
def parseInput(row):
    import re
    
    pattern = re.compile( r'\(\'([a-z])\', ([0-9])\)' )
    row_split = pattern.split(row)
    return ( row_split[1], int(row_split[2]) )

In [94]:
data_key_reread = sc.textFile(
    '../../data/RDD_example/PySpark_data_key.txt').map(parseInput)

In [95]:
data_key_reread.collect()

[('a', 4), ('b', 3), ('c', 2), ('a', 8), ('d', 2), ('b', 1), ('d', 3)]